Our goal for this question is to find all the actors **besides** Sylvester Stallone who appeared in a _Rocky_ movie.  We will define a _Rocky_ movie to be a movie:

* in which Sylvester Stallone appeared and
* whose `Title` attribute **contains** the word *Rocky*

We'll get started by writing a query to find _Rocky_ movies, displaying the `Title`, `YearReleased`, `FirstName`, `LastName`, and `Role` attributes to check our work.

In [1]:
SELECT Title, YearReleased, FirstName, LastName, "Role"
from imdb.People
join imdb.Principals
  on People.PersonID = Principals.PersonID
join imdb.Movies
  on Movies.TitleID = Principals.TitleID
where FirstName = 'Sylvester'
  and LastName = 'Stallone'
  and IsActor = 1
  and Title like 'Rocky%'

(6 rows affected)

Total execution time: 00:00:01.542

Title,YearReleased,FirstName,LastName,Role
Rocky,1976,Sylvester,Stallone,Rocky
Rocky II,1979,Sylvester,Stallone,Rocky Balboa
Rocky III,1982,Sylvester,Stallone,Rocky Balboa
Rocky IV,1985,Sylvester,Stallone,Rocky Balboa
Rocky V,1990,Sylvester,Stallone,Rocky
Rocky Balboa,2006,Sylvester,Stallone,Rocky Balboa


Now that we have the list of movies, we want to actors who appeared in **that set** of `Movie`s. We'll use a **common table expression** to define a table named `RockyMovies` to make this happen.  At this stage, we'll just select everything from `RockyMovies` to demonstrate the use of CTEs.

Here's a copy of the syntax for a CTE from the Microsoft documentation.

```
[ WITH <common_table_expression> [ ,...n ] ]

<common_table_expression>::=
    expression_name [ ( column_name [ ,...n ] ) ]
    AS
    ( CTE_query_definition )

```

In [2]:
WITH RockyMovies
AS
(
SELECT Title, YearReleased, FirstName, LastName, "Role"
from imdb.People
join imdb.Principals
  on People.PersonID = Principals.PersonID
join imdb.Movies
  on Movies.TitleID = Principals.TitleID
where FirstName = 'Sylvester'
  and LastName = 'Stallone'
  and IsActor = 1
  and Title like 'Rocky%'
)
select * from RockyMovies

(6 rows affected)

Total execution time: 00:00:00.025

Title,YearReleased,FirstName,LastName,Role
Rocky,1976,Sylvester,Stallone,Rocky
Rocky II,1979,Sylvester,Stallone,Rocky Balboa
Rocky III,1982,Sylvester,Stallone,Rocky Balboa
Rocky IV,1985,Sylvester,Stallone,Rocky Balboa
Rocky V,1990,Sylvester,Stallone,Rocky
Rocky Balboa,2006,Sylvester,Stallone,Rocky Balboa


Now we want to find all actors in those movies.  We'll need to modify the attributes included in `RockyMovies` to include the `TitleID`.

We'll include only the `TitleID`, `Title`, and `YearReleased` attributes, and rename the `TitleID` attribute `RockyMovieID` to see how that works.  Here's the syntax again.

````
[ WITH <common_table_expression> [ ,...n ] ]

<common_table_expression>::=
    expression_name [ ( column_name [ ,...n ] ) ]
    AS
    ( CTE_query_definition )
````

In [5]:
WITH RockyMovies
AS
(
SELECT Movies.TitleID, Title, YearReleased
from imdb.People
join imdb.Principals
  on People.PersonID = Principals.PersonID
join imdb.Movies
  on Movies.TitleID = Principals.TitleID
where FirstName = 'Sylvester'
  and LastName = 'Stallone'
  and IsActor = 1
  and Title like 'Rocky%'
)
select Title, YearReleased, FirstName, LastName, "Role" 
from imdb.People
join imdb.Principals
  on People.PersonID = Principals.PersonID
join RockyMovies
  on RockyMovies.TitleID = Principals.TitleID
ORDER by YearReleased, CreditNumber

(59 rows affected)

Total execution time: 00:00:00.190

Title,YearReleased,FirstName,LastName,Role
Rocky,1976,Sylvester,Stallone,Rocky
Rocky,1976,Talia,Shire,Adrian
Rocky,1976,Burt,Young,Paulie
Rocky,1976,Carl,Weathers,Apollo
Rocky,1976,John,Avildsen,NULL
Rocky,1976,Robert,Chartoff,NULL
Rocky,1976,Irwin,Winkler,NULL
Rocky,1976,Bill,Conti,NULL
Rocky,1976,James,Crabe,NULL
Rocky,1976,Scott,Conrad,NULL


Now we need to eliminate Sly from the results.  We'll do it by writing 3 CTEs:

- the first will be called **Sly** and will contain just the `PersonID` attribute for _Sylvester Stallone_
- the second will be our `RockyMovies` CTE which uses `Sly` to find the Rocky movies
- the 3rd will be all the people who appeared in a Rocky movie

We will then find all the people aren't Sylvester Stallone  from the 3<sup>rd</sup> CTE.

Again, here's the CTE syntax.

```
[ WITH <common_table_expression> [ ,...n ] ]

<common_table_expression>::=
    expression_name [ ( column_name [ ,...n ] ) ]
    AS
    ( CTE_query_definition )

```

In [8]:
with Sly (SlyID) as 
(
    select PersonID
    from imdb.People
    where FirstName = 'Sylvester'
      and Lastname = 'Stallone'
),
RockyMovies AS
(
SELECT Movies.TitleID, Title, YearReleased
from imdb.Principals
join Sly
  on SlyID = PersonID
join imdb.Movies
  on Movies.TitleID = Principals.TitleID
where Title like 'Rocky%'
),
PeopleInRockyMovies AS
(
select People.PersonID, Title, YearReleased, FirstName, LastName, "Role", CreditNumber
from imdb.People
join imdb.Principals
  on People.PersonID = Principals.PersonID
join RockyMovies
  on RockyMovies.TitleID = Principals.TitleID
)
select *
from PeopleInRockyMovies, Sly
where PeopleInRockyMovies.PersonID != SlyID
Order by YearReleased, CreditNumber

(53 rows affected)

Total execution time: 00:00:00.023

PersonID,Title,YearReleased,FirstName,LastName,Role,CreditNumber,SlyID
nm0001735,Rocky,1976,Talia,Shire,Adrian,2,nm0000230
nm0949350,Rocky,1976,Burt,Young,Paulie,3,nm0000230
nm0001835,Rocky,1976,Carl,Weathers,Apollo,4,nm0000230
nm0000814,Rocky,1976,John,Avildsen,NULL,5,nm0000230
nm0153590,Rocky,1976,Robert,Chartoff,NULL,6,nm0000230
nm0005563,Rocky,1976,Irwin,Winkler,NULL,7,nm0000230
nm0006015,Rocky,1976,Bill,Conti,NULL,8,nm0000230
nm0185583,Rocky,1976,James,Crabe,NULL,9,nm0000230
nm0175722,Rocky,1976,Scott,Conrad,NULL,10,nm0000230
nm0001735,Rocky II,1979,Talia,Shire,Adrian,2,nm0000230
